---
toc: false
comments: true
layout: post
title: Team Teach - Full Stack and User Profile With Commentary
description: CRUD backend and frontend, Authentication
type: hacks
courses: { compsci: {week: 19} }
---

# CRUD

## What is CRUD?

CRUD stands for:

C-reate: Involves the creation of addition of data to a database. In essence, it "create"s entries in databases.

R-ead: Involves the retrieval, querying, or "read"ing of existing data from a database.

U-pdate: Involves the modification or "update"ing of existing data in a database. Operation of changing the values of specific fields in a system. 

D-elete: Involves the removal or "delete"ion of existing data from a database. It's the operation of eliminating information from a database.

In the past, each function has had its own link to fetch from, but with CRUD all four functions will be under one link, differentiated by the request type most suited for it.
Create: POST request, as create operation needs to sent data back to the backend, best suited by post request
Read: GET request, as read operation is simple and requires no data to be sent back
Update: PUT request, as update requires the user ID of the user being updated, and this can be modified in the URL which PUT requests allow
Delete: DELETE request, self explanatory
This is simpler and makes fetching less confusing, as everything is done under one link. 

## Authentication
- How users will authenticate to be able to access function that require a token
- POST request is sent to backend
    - payload contains user ID and password
- Backend compares user ID and password, and returns with token if successful
- Note that authentication and authorization are two different things
    - Authentication is identifying who you are, client saying hi to backend
    - Authorization is getting permission to view something, website letting client know that they can view something



#### Frontend

- This section of the frontend(the code below) will mainly be the connection between frontend and backend in which the frontend calls to the api endpoint in the backend for acsess to database and to perform different CRUD operations


In [ ]:
function login_user(){
    // Set Authenticate endpoint
    const url = 'http://127.0.0.1:8086/api/users/authenticate';

    // Set the body of the request to include login data from the DOM
    const body = {
        uid: document.getElementById("uid").value,
        password: document.getElementById("password").value,
    };

    // Change options according to Authentication requirements
    const authOptions = {
        mode: 'cors', // no-cors, *cors, same-origin
        credentials: 'include', // include, same-origin, omit
        headers: {
            'Content-Type': 'application/json',
        },
        method: 'POST', // Override the method property
        cache: 'no-cache', // Set the cache property
        body: JSON.stringify(body)
    };

    // Fetch JWT
    fetch(url, authOptions)
    .then(response => {
        // handle error response from Web API
        if (!response.ok) {
            const errorMsg = 'Login error: ' + response.status;
            console.log(errorMsg);
            alert("Failed Authentication: Credentials Incorrect")
            return;
        }
        // Success!!!
        // Redirect to the database page
        window.location.href = "/csp-blog//log/2024/01/30/Users.html";
    })
    // catch fetch errors (ie ACCESS to server blocked)
    .catch(err => {
        console.error(err);
    });
}

// Attach login_user to the window object, allowing access to form action
window.login_user = login_user;

### Line(s) by Line tutotorial explanation of code above



In [ ]:
function login_user(){
    // Set Authenticate endpoint
    const url = 'http://127.0.0.1:8086/api/users/authenticate';

    // Set the body of the request to include login data from the DOM
    const body = {
        uid: document.getElementById("uid").value,
        password: document.getElementById("password").value,
    };
    

### Analysis

- First it defines the function as login_user and this is the login button function essentially that is called through your markdown/html code

- The const url is set to be the authenticate URL, which you used in Postman(with this url we will be logged in and obtain a cookie, which will ensure we will have all the acsess we need to things such as a datatable of all users)
    
- Afterward in the const body variable we see uid: the document.getElementbyID and also the same for password. These 2 lines are just acsessing the text entered in our markdown/html textbox. 

In [ ]:
const authOptions = {
        mode: 'cors', // no-cors, *cors, same-origin
        credentials: 'include', // include, same-origin, omit
        headers: {
            'Content-Type': 'application/json',
        },
        method: 'POST', // Override the method property
        cache: 'no-cache', // Set the cache property
        body: JSON.stringify(body)
    };

### Analysis

- It defines a variable called auth options

- Some important things to note in here is the "method:" in the middle of this segment and we can see it's post(the reason it is post is because we use 'post' method for authentication as we need to make a cookie)

- Another thing we can notice is the "body: JSON.stringify(body)", which will also add the JSON data to the variable(the json data is things such as uid/password)

In [ ]:
fetch(url, authOptions)
    .then(response => {
        // handle error response from Web API
        if (!response.ok) {
            const errorMsg = 'Login error: ' + response.status;
            console.log(errorMsg);
            alert("Failed Authentication: Credentials Incorrect")
            return;
        }
        // Success!!!
        // Redirect to the database page
        window.location.href = "/csp-blog//log/2024/01/30/Users.html";
    })
    // catch fetch errors (ie ACCESS to server blocked)
    .catch(err => {
        console.error(err);
    });

### Analysis

- We use the url and authoptions variables to finally fetch the data from the backend and add to it too

- In the if statement we notice it says !reponse.ok and this essentially will make sure no errors and if there is one it will log it to console

- After the if statement(so this means no errors) the window.location.href is equal to a specific link and this will essentially navigate the user to the datatable page(the datatable will have a list of all users), but note we can also change this to navigate to any page we want

## Backend 

In [ ]:
from werkzeug.security import generate_password_hash, check_password_hash
# update password, this is conventional setter
def set_password(self, password):
    """Create a hashed password."""
    self._password = generate_password_hash(password, "pbkdf2:sha256", salt_length=10)

# check password parameter versus stored/encrypted password
def is_password(self, password):
    """Check against hashed password."""
    result = check_password_hash(self._password, password)
    return result

In these lines here we are encrypting the password into the database so if anyone obtains acsess they will not be able to login to other users account or hack them. 
- Prevents somebody from viewing passwords, even if they were somehow able to access the database/code

In [ ]:
def post(self):
            try:
                body = request.get_json()
                if not body:
                    return {
                        "message": "Please provide user details",
                        "data": None,
                        "error": "Bad request"
                    }, 400
                ''' Get Data '''
                uid = body.get('uid')
                if uid is None:
                    return {'message': f'User ID is missing'}, 400
                password = body.get('password')
                
                ''' Find user '''
                user = User.query.filter_by(_uid=uid).first()
                if user is None or not user.is_password(password):
                    return {'message': f"Invalid user id or password"}, 400
                if user:
                    try:
                        token = jwt.encode(
                            {"_uid": user._uid},
                            current_app.config["SECRET_KEY"],
                            algorithm="HS256"
                        )
                        resp = Response("Authentication for %s successful" % (user._uid))
                        resp.set_cookie("jwt", token,
                                max_age=3600,
                                secure=True,
                                httponly=False,
                                path='/',
                                samesite='None'  # This is the key part for cross-site requests

                                # domain="frontend.com"
                                )
                        return resp
                    except Exception as e:
                        return {
                            "error": "Something went wrong",
                            "message": str(e)
                        }, 500
                return {
                    "message": "Error fetching auth token!",
                    "data": None,
                    "error": "Unauthorized"
                }, 404
            except Exception as e:
                return {
                        "message": "Something went wrong!",
                        "error": str(e),
                        "data": None
                }, 500

The above code compares the entered user password to the on located in the database. It first ensures that a user ID and password has been provided, then compares it and if successful returns a authentication success message. This is accompanied by a token which contains the key to the website encoded, meaning that nobody can copy the key and create a token to get access.

## Create
#### Frontend

In [ ]:
import { uri, options } from '{{site.baseurl}}/assets/js/api/config.js';
//
//    const url = uri + '/api/users/authenticate';
//    const body = {
//            // name: document.getElementById("name").value,
//            uid: "toby",
//            password: "123toby"
//            // dob: document.getElementById("dob").value
//        };
//    const authOptions = {
//            ...options, // This will copy all properties from options
//            method: 'POST', // Override the method property
//            cache: 'no-cache', // Set the cache property
//            body: JSON.stringify(body)//       };
//    fetch(url, authOptions)
    function login_user(){
        // Set Authenticate endpoint
        const url = uri + '/api/users/';

        // Set the body of the request to include login data from the DOM
        const body = {
            name: document.getElementById("name").value,
            uid: document.getElementById("uid").value,
            password: document.getElementById("password").value,
            dob: document.getElementById("dob").value
        };

        // Change options according to Authentication requirements
        const authOptions = {
            ...options, // This will copy all properties from options
            method: 'POST', // Override the method property
            cache: 'no-cache', // Set the cache property
            body: JSON.stringify(body)
        };

        // Fetch JWT
        fetch(url, authOptions)
        .then(response => {
            // handle error response from Web API
            if (!response.ok) {
                const errorMsg = 'Login error: ' + response.status;
                console.log(errorMsg);
                return;
            }
            // Success!!!
            // Redirect to home page or wherever you need to
            window.location.href = "{{site.baseurl}}/data/database";
        })
        // catch fetch errors (ie ACCESS to server blocked)
        .catch(err => {
            console.error(err);
        });
    }

    // Attach login_user to the window object, allowing access to form action
    window.login_user = login_user;

The above code sends a POST request to the backend to create a user. It sends along a payload with the information for creating a user. When it recieves a response, it checks if the response is not standard using "!response.ok" and logs an error if so. If the response is proper it then redirects the user to another page, in this case it is a database page but really is whatever you need.

#### Backend

In [ ]:
def post(self): # Create method
    ''' Read data for json body '''
    body = request.get_json()
    
    ''' Avoid garbage in, error checking '''
    # validate name
    name = body.get('name')
    if name is None or len(name) < 2:
        return {'message': f'Name is missing, or is less than 2 characters'}, 400
    # validate uid
    uid = body.get('uid')
    if uid is None or len(uid) < 2:
        return {'message': f'User ID is missing, or is less than 2 characters'}, 400
    # look for password and dob
    password = body.get('password')
    dob = body.get('dob')

    ''' #1: Key code block, setup USER OBJECT '''
    uo = User(name=name, 
                uid=uid)
    
    ''' Additional garbage error checking '''
    # set password if provided
    if password is not None:
        uo.set_password(password)
    # convert to date type
    if dob is not None:
        try:
            uo.dob = datetime.strptime(dob, '%Y-%m-%d').date()
        except:
            return {'message': f'Date of birth format error {dob}, must be mm-dd-yyyy'}, 400
    
    ''' #2: Key Code block to add user to database '''
    # create user in database
    user = uo.create()
    # success returns json of user
    if user:
        return jsonify(user.read())
    # failure returns error
    return {'message': f'Processed {name}, either a format error or User ID {uid} is duplicate'}, 400

- This is the post method or the create method and what it does is add users to the database. This method is vital for activities in websites such as creating an account. In the line uo=User(name=name,uid,active_classes,archieved_classes=archieved_classes), we can see the user being created through its specializec class. 
- The @token_required line is removed to ensure that anybody can create a user
- Because of this, the current_user argument is removed as no user is being passed through when a user is created.

## Read
#### Frontend
This example read code sends a GET request to the backend. Note that read doesn't necessarily mean returning the entire database, but simply reading already existing information from the database. Note that the javaScript below creates a table with the retrieved information but other things can be done with it.

In [ ]:
// uri variable and options object are obtained from config.js
  import { uri, options } from '/teacher_portfolio/assets/js/api/config.js';

  // Set Users endpoint (list of users)
  const url = uri + '/api/users/';

  // prepare HTML result container for new output
  const resultContainer = document.getElementById("result");

  // fetch the API
  fetch(url, options)
    // response is a RESTful "promise" on any successful fetch
    .then(response => {
      // check for response errors and display
      if (response.status !== 200) {
          const errorMsg = 'Database response error: ' + response.status;
          console.log(errorMsg);
          const tr = document.createElement("tr");
          const td = document.createElement("td");
          td.innerHTML = errorMsg;
          tr.appendChild(td);
          resultContainer.appendChild(tr);
          return;
      }
      // valid response will contain JSON data
      response.json().then(data => {
          console.log(data);
          for (const row of data) {
            // tr and td build out for each row
            const tr = document.createElement("tr");
            const name = document.createElement("td");
            const id = document.createElement("td");
            const age = document.createElement("td");
            // data is specific to the API
            name.innerHTML = row.name; 
            id.innerHTML = row.uid; 
            age.innerHTML = row.age; 
            // this builds td's into tr
            tr.appendChild(name);
            tr.appendChild(id);
            tr.appendChild(age);
            // append the row to table
            resultContainer.appendChild(tr);
          }
      })
  })
  // catch fetch errors (ie ACCESS to server blocked)
  .catch(err => {
    console.error(err);
    const tr = document.createElement("tr");
    const td = document.createElement("td");
    td.innerHTML = err + ": " + url;
    tr.appendChild(td);
    resultContainer.appendChild(tr);
  });

### Analysis
- In this javascript code we are obtaining all of the information from the database using the R-Read method or the get method from the backend and with this we are able to make a table of all the user ids similar to the one on the nighthawkcoders teacher website. in the fet(uril,option) it is similar to the previous javascript code but instead the options defined seems to use an imported definition. In the fetch of data, we can see in the valid response that we define four variables, which is the actual empty row, the names, the ids, and finally the ages. We then after setting each of these variables add it to the row, which is then added to the table. With this code we are able to visualize the whole database in an easy table format. 





#### Backend
The below code returns all users from the database in as JSON when fetched.

In [ ]:
@token_required
def get(self, current_user): # Read Method
    users = User.query.all()    # read/extract all users from database
    json_ready = [user.read() for user in users]  # prepare output in json
    return jsonify(json_ready)  # jsonify creates Flask response object, more specific to APIs than json.dumps

## Analysis

This code segment is extremely short but in the code above we can see the get method being defined. Above the actual function defining, we can see the @token_required, which means that it will require the user to be logged in if they want to view the data from the database. In the actual code, we obtain all users from the database and format it in jSON so it can be returned to the frontend and be easily visualized. 

## Put
#### Frontend
The following code sends a put request to backend to update a user based on the user's UID.

In [ ]:
function update_user(){
      if (document.getElementById("password").value != document.getElementById("confirmpassword").value) {
        alert("Error: Passwords do not match.");
        return;
      }
      const body = {
        uid: document.getElementById("uid").value,
        password: document.getElementById("password").value,
        name: document.getElementById("name").value,
      };
      const AuthOptions = {
                  mode: 'cors', // no-cors, *cors, same-origin
                  credentials: 'include', // include, same-origin, omit
                  headers: {
                      'Content-Type': 'application/json',
                  },
                  method: 'PUT', // Override the method property
                  cache: 'no-cache', // Set the cache property
                  body: JSON.stringify(body)
              };
        // fetch the API
        fetch(url, AuthOptions)
          // response is a RESTful "promise" on any successful fetch
          .then(response => {
            // check for response errors and display
            if (response.status !== 200) {
                window.location.href = "/csp-blog/403.html";
            }
            // valid response will contain JSON data
            response.json().then(data => {
              // insert whatever code you want here
            })
        })
        // catch fetch errors (ie ACCESS to server blocked)
        .catch(err => {
          console.log(err)
        });
    }
    // Attach login_user to the window object, allowing access to form action
    window.update_user = update_user;

## Code Analysis

For an overview of the code above, it is a javascript function, which updates user information and user profile. 

We first start out by ensuring that an changes are password-protected so the user must enter their password to make any changes to their profile. We then define the body variable, which has the entered UID, Password, and name of the profile. The next Authoptions variables is another thing we have seen before in previous api-calling codes but the main difference here is that it has method: 'PUT', which indicates that this is an update function or the U in CRUD. We then do the normal checking of if theres no errors, but now we can customize any information that we want the user to be able to change.  

#### Backend

In [ ]:
@token_required
def put(self, current_user):
    body = request.get_json() # get the body of the request
    uid = body.get('uid') # get the UID (Know what to reference)
    name = body.get('name') # get name (to change)
    password = body.get('password') # get password (to change)
    users = User.query.all() # get users
    for user in users:
        if user.uid == uid: # find user with matching uid
            user.update(name,'',password) # update info
    return f"{user.read()} Updated"

### Code Analysis

In the code above we have the put function, or the UPDATE method, which will allow us to update the user profile within the database. We first obtain information of details of the user that we want to change such as username, name, and password. We then find the user using their username. and are able to update only the name and password here. 

## Delete
#### Frontend
Similarly to put, this code removes the appropriate user from the database

In [ ]:
function delete_user(){
    const body = {
      uid: document.getElementById("uid").value,
    };
    const AuthOptions = {
                mode: 'cors', // no-cors, *cors, same-origin
                credentials: 'include', // include, same-origin, omit
                headers: {
                    'Content-Type': 'application/json',
                },
                method: 'DELETE', // Override the method property
                cache: 'no-cache', // Set the cache property
                body: JSON.stringify(body)
            };
      // fetch the API
      fetch(url, AuthOptions)
        // response is a RESTful "promise" on any successful fetch
        .then(response => {
          // check for response errors and display
          if (response.status !== 200) {
              window.location.href = "/csp-blog/403.html";
          }
          // valid response will contain JSON data
          response.json().then(data => {
            window.location.href = "/csp-blog//log/2024/01/30/Users.html";
          })
      })
      // catch fetch errors (ie ACCESS to server blocked)
      .catch(err => {
        console.log(err)
      });
  }
  window.delete_user = delete_user;

#### Backend

In [ ]:
def delete(self, id):  # Delete method
    #Find user by ID 
    user = User.query.get(id)
    if user is None:
        return {'message': f'User with ID {id} not found'}, 404

    ''' Delete user '''
    user.delete()
    return {'message': f'User with ID {id} has been deleted'}

## Analysis of Delete 1 (ID Version)

- This code is the delete function and allows us to do the action of removing a user profile and removing a user from the database. In this code, it is able to delete the user using the database ID, which is DIFFERENT from the uid or user id and this value is a number automatically assigned by the database. 
- The code finds the user by its ID and then deletes it.

In [ ]:
@token_required
def delete(self, current_user):
    body = request.get_json()
    uid = body.get('uid')
    users = User.query.all()
    for user in users:
        if user.uid == uid:
            user.delete()
    return jsonify(user.read())

## Delete #2 (UID Version)

- This code is similar to version 1 but is different as it deletes based on the uid or username. For example, if I enter the user ID as toby, it will delete the profile of toby instead of I enter the database index of 1. In here, we obtain the JSON of the userID information and then search throughout the database for values that match the inputted USERID and if it matches then we must delete the user that matches it. 
- Disadvantage: Anybody can still send a request and delete anybody, there is no guarantee somebody is trying to delete themself

In [ ]:
   
@token_required
def delete(self, current_user):
# body = request.get_json()
    token = request.cookies.get("jwt")
    cur_user = jwt.decode(token, current_app.config["SECRET_KEY"], algorithms=["HS256"])['_uid']
    users = User.query.all()
    for user in users:
        if user.uid==cur_user: # modified with the and user.id==cur_user so random users can't delete other ppl
            user.delete()
    return jsonify(user.read())

## Delete 3 (Cookie Version)

- This code is similar to the first two but is a much bigger jump as now it does not take any input on how to delete the user. Instead of taking input, such as username or database index, we now make it such that the user can only delete their own account and not others. This implementation has the advatage of the fact that a user shouldn't be able to delete other people's account but other implementations defintely are still accurate and work. For some more in-depth analysis, the cur_user variable includes information of the current user trying to call the delete method and the next few lines search the database and check which one is the current user's spot, so it can then delete it. 
- Disadvantage: What if a database admin wants to delete another user?


<script>
    function deleteUser() {
        // You can add your logic for deleting the user here
        console.log("in function");
        const url = 'http://127.0.0.1:8086/api/users/';
        const options = {
            mode: 'cors', // no-cors, cors, same-origin
            credentials: 'include', // include, same-origin, omit
            headers: {
                'Content-Type': 'application/json'
            },
            method: 'DELETE', // Override the method property
            cache: 'no-cache', // Set the cache property
        };
        fetch(url, options)
        // response is a RESTful "promise" on any successful fetch
        .then(response => {
            // check for response errors and display
            if (response.status !== 200) {
                const errorMsg = 'Database response error: ' + response.status;
                window.location.href = "http://127.0.0.1:4200/student/2024/01/31/403error.html";
                console.log(errorMsg);
                const tr = document.createElement("tr");
                const td = document.createElement("td");
                td.innerHTML = errorMsg;
                tr.appendChild(td);
                resultContainer.appendChild(tr);
                return;
            }
            // valid response will contain JSON data
            response.json().then(data => {
                console.log("worked");
                console.log(data);
                 window.location.href = "http://127.0.0.1:4200/student/2024/01/30/DataTable.html";
            })
            // catch fetch errors (i.e., ACCESS to server blocked)
            .catch(err => {
                console.error(err);
                const tr = document.createElement("tr");
                const td = document.createElement("td");
                td.innerHTML = err + ": " + url;
                tr.appendChild(td);
                resultContainer.appendChild(tr);
            ;
            });
        });

        
    }
        
</script>